In [41]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.integrators import LangevinIntegrator
from tqdm import tqdm_notebook
from simtk.openmm import app

In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [3]:
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/4/"
resid = '493'
old_aa_name = 'GLN'
new_aa_name = 'ALA'
sim_number = 3

In [4]:
if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1

length = 1
move_length = 3
timestep = 2
radius = 0.3



In [5]:
# Load rhtf 
i = os.path.basename(os.path.dirname(dir))
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))



INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/4/4_complex_0.pickle


In [6]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
# query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(args.resid)].atoms]
for res in htf.hybrid_topology.residues:
    if res.resSeq == int(resid) and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
_logger.info(f"rest atoms {rest_atoms}")
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system



INFO:root:Generating REST factory
INFO:root:radius:0.3 nm
INFO:root:query indices [2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 200474, 200475, 200476, 200477]
<ipython-input-6-2319932805d9>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
INFO:root:rest atoms [1828, 1831, 1844, 1845, 1854, 1856, 1857, 1864, 1905, 2467, 2468, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489, 2490, 2491, 249

In [31]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
#     feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))



In [14]:
# # Set up sampler
# # _logger.setLevel(logging.DEBUG)
# # _logger.info("About to start repex")
# # print(f"move steps: {int((move_length*1000)/timestep)}")
# # print(f"timestep: {timestep} fs")
# timestep = 4
# move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
# simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)



In [32]:
for i in range(12):
    print(thermodynamic_state_list[i].reduced_potential(sampler_state_list[i]))

TypeError: object of type 'float' has no len()

In [17]:
# # Run t-repex
# reporter_file = os.path.join(dir, f"{i}_{phase}_{name.lower()}_{length}ns.nc")
# reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
# simulation.create(thermodynamic_states=thermodynamic_state_list[:-1],
#                   sampler_states=sampler_state_list[:-1],
#                   storage=reporter)
# simulation.run()


DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/14/4/11_complex_gln_1ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b4956a2fbb0>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b498d03c310>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b498cff2c10>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b498cff2ee0>


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  2980173B | 2910.325KB | 2.842MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    8.764s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b498cff5040>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b498cff5310>
DEBUG:openmmtools.utils:Storing sampler states took    0.055s
DEBUG:openmmtools.utils:Writing iteration information to storage took    1.166s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    1.032s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b4956a2fbb0>>
DEBUG:openmmtools.multistate.multist

SimulationNaNError: Propagating replica 0 at state 10 resulted in a NaN!
The state of the system and integrator before the error were saved in /data/chodera/zhangi/perses_benchmark/neq/14/4/nan-error-logs

In [33]:
# Define simulation parameters
temperature = 298 * unit.kelvin
collision_rate = 1.0 / unit.picoseconds
nsteps = 100 
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
integrator = LangevinIntegrator(temperature, collision_rate, timestep)

system = htf.hybrid_system
positions = htf.hybrid_positions

platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = thermodynamic_state_list[0].create_context(integrator, platform=platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)



In [34]:
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=463322164233.96497, unit=kilojoule/mole)

In [35]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [36]:
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=462868880899.6945, unit=kilojoule/mole)

In [37]:
# Run equilibration
for step in tqdm_notebook(range(nsteps)):
    integrator.step(1)

<ipython-input-37-951c7b78bb29>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm_notebook(range(nsteps)):


  0%|          | 0/100 [00:00<?, ?it/s]

In [38]:
context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=nan, unit=kilojoule/mole)

In [42]:
app.PDBFile.writeFile(htf._topology_proposal.old_topology, htf.old_positions(htf.hybrid_positions), open(os.path.join(dir, "complex_old.pdb"), "w"), keepIds=True)

In [44]:
phase = 'apo'
i = 4
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
apo_htf = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/4/4_apo_0.pickle


In [45]:
app.PDBFile.writeFile(apo_htf._topology_proposal.old_topology, apo_htf.old_positions(apo_htf.hybrid_positions), open(os.path.join(dir, "apo_old.pdb"), "w"), keepIds=True)

In [48]:
phase = 'apo'
i = 5
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/5/"
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf_5 = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/5/5_apo_0.pickle


In [49]:
app.PDBFile.writeFile(htf_5._topology_proposal.old_topology, htf_5.old_positions(htf_5.hybrid_positions), open(os.path.join(dir, "apo_old.pdb"), "w"), keepIds=True)

In [50]:
phase = 'apo'
i = 3
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/3/"
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf_3 = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/3/3_apo_0.pickle


In [51]:
app.PDBFile.writeFile(htf_3._topology_proposal.old_topology, htf_3.old_positions(htf_3.hybrid_positions), open(os.path.join(dir, "apo_old.pdb"), "w"), keepIds=True)


In [52]:
phase = 'apo'
i = 2
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/2/"
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf_2 = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/2/2_apo_0.pickle


In [53]:
app.PDBFile.writeFile(htf_2._topology_proposal.old_topology, htf_2.old_positions(htf_2.hybrid_positions), open(os.path.join(dir, "apo_old.pdb"), "w"), keepIds=True)


In [54]:
phase = 'apo'
i = 1
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/1/"
path = os.path.join(dir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf_1 = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/1/1_apo_0.pickle


In [55]:
app.PDBFile.writeFile(htf_1._topology_proposal.old_topology, htf_1.old_positions(htf_1.hybrid_positions), open(os.path.join(dir, "apo_old.pdb"), "w"), keepIds=True)


In [57]:
phase = 'apo'
i = 1
dir = "/data/chodera/zhangi/perses_benchmark/neq/14/1/"
path = os.path.join(dir, f"{i}_{phase}.pickle")
_logger.info(f"path: {path}")
htf_1_vanilla = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/1/1_apo.pickle


In [58]:
app.PDBFile.writeFile(htf_1_vanilla._topology_proposal.old_topology, htf_1_vanilla.old_positions(htf_1_vanilla.hybrid_positions), open(os.path.join(dir, "apo_old_vanilla.pdb"), "w"), keepIds=True)


In [60]:
import mdtraj as md

In [62]:
traj = md.Trajectory(htf_1_vanilla.hybrid_positions, htf_1_vanilla.hybrid_topology)
traj.save(os.path.join(dir, "apo_hybrid_vanilla.pdb"))

In [63]:
htf_1_vanilla._atom_classes['unique_old_atoms']

{1640, 1641, 1642, 1643, 1644}

In [64]:
for atom in htf_1_vanilla._topology_proposal.old_topology.atoms():
    if atom.index in {1640, 1641, 1642, 1643, 1644}:
        print(atom)

<Atom 1640 (CG) of chain 0 residue 107 (ASN)>
<Atom 1641 (OD1) of chain 0 residue 107 (ASN)>
<Atom 1642 (ND2) of chain 0 residue 107 (ASN)>
<Atom 1643 (HD21) of chain 0 residue 107 (ASN)>
<Atom 1644 (HD22) of chain 0 residue 107 (ASN)>
